# <b>  API Data ETL

In [36]:
import requests
from sqlalchemy import create_engine, Table, Column, MetaData, String, Integer
from sqlalchemy.exc import SQLAlchemyError

## <b>Extract Data

In [37]:
# Replace 'YOUR_API_KEY' with your actual OpenWeather API key
api_key = "Your_API_Key"
city = "London"  # You can change the city to any location of your choice
weather_api_url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"

def fetch_data(url):
    """Fetch data from the API."""
    try:
        # Make a GET request to the API
        response = requests.get(url)
        # Raise an exception for HTTP errors
        response.raise_for_status()
        # Return the JSON response
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

## <b> Transform Data

In [38]:
def transform_data(data):
    """Transform JSON data into a tabular format."""
    if data:
        try:
            # Extract relevant data from the JSON response
            transformed_data = {
                'temperature': data['main']['temp'],  # Temperature in Celsius
                'humidity': data['main']['humidity'],  # Humidity in percentage
                'description': data['weather'][0]['description']  # Weather description
            }
            return transformed_data
        except KeyError as e:
            print(f"Error transforming data: Missing key {e}")
            return None
    else:
        return None

## <b> Load Data

In [39]:
def load_data(data):
    """Load data into a SQLite database."""
    if data:
        try:
            # Create a SQLite engine
            engine = create_engine('sqlite:///weather_data.db', echo=True)
            # Create a metadata object
            metadata = MetaData()

            # Define a table schema
            weather_table = Table('weather', metadata,
                                  Column('id', Integer, primary_key=True, autoincrement=True),
                                  Column('temperature', String),
                                  Column('humidity', String),
                                  Column('description', String))

            # Create the table in the database
            metadata.create_all(engine)

            # Insert data into the table
            with engine.connect() as connection:
                insert_stmt = weather_table.insert().values(
                    temperature=data['temperature'],
                    humidity=data['humidity'],
                    description=data['description']
                )
                # Execute the insert statement
                connection.execute(insert_stmt)
                connection.commit()  # Explicitly commit the transaction
                print("Data loaded successfully.")
        except SQLAlchemyError as e:
            print(f"Error loading data: {e}")
    else:
        print("No data to load.")


## <b>Main Function

In [40]:
# Run the ETL process
weather_data = fetch_data(weather_api_url)  # Extract data from the API
transformed_weather_data = transform_data(weather_data)  # Transform the data
load_data(transformed_weather_data)  # Load the data into the database

2024-06-28 23:40:50,651 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-28 23:40:50,652 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("weather")
2024-06-28 23:40:50,653 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-28 23:40:50,656 INFO sqlalchemy.engine.Engine COMMIT
2024-06-28 23:40:50,659 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-28 23:40:50,660 INFO sqlalchemy.engine.Engine INSERT INTO weather (temperature, humidity, description) VALUES (?, ?, ?)
2024-06-28 23:40:50,661 INFO sqlalchemy.engine.Engine [generated in 0.00232s] (19.9, 51, 'few clouds')
2024-06-28 23:40:50,664 INFO sqlalchemy.engine.Engine COMMIT
Data loaded successfully.
